In [1]:
import pandas as pd
import db_access

In [2]:
def get_raw_data(limit=None):
    engine = db_access.get_engine()
    query = "select * from usd_jpy"
    if (limit is not None and type(limit) is int):
        query = query + "limit " + str(limit)
    return pd.read_sql("SELECT * FROM usd_jpy", engine)

In [19]:
df = get_raw_data()
df = df[["datetime","open","high","low","close","volume"]].copy()
df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']
df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.reset_index().set_index('Datetime')
df.head()

,index,Open,High,Low,Close,Volume
Datetime,,,,,,
2020-01-30 04:30:00,0,108.893,108.914,108.888,108.912,92
2020-01-30 04:35:00,1,108.914,108.923,108.909,108.921,121
2020-01-30 04:40:00,2,108.920,108.932,108.917,108.927,131
2020-01-30 04:45:00,3,108.926,108.928,108.904,108.908,143
2020-01-30 04:50:00,4,108.907,108.908,108.896,108.900,99


In [21]:
import talib as ta

In [22]:
def get_bband(close, timeperiod=20, nbdevup=3, nbdevdn=3, matype=0):
    gain = pd.DataFrame(close)
    gain.columns = ['close']
    upper, middle, lower = ta.BBANDS(gain.close, timeperiod, nbdevup , nbdevdn, matype)
    gain['bb_upper'] = upper
    gain['bb_lower'] = lower
    return gain['bb_upper'], gain['bb_lower']

In [36]:
def get_macd(close, fastperiod=6, slowperiod=13, signalperiod=4):
    macd, macdsignal, macdhist = ta.MACD(close, fastperiod=fastperiod, slowperiod=slowperiod, signalperiod=signalperiod)
    return macd, macdsignal

In [25]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [54]:
# https://bollinger-band.org/entry25.html
class MacdBbandStrategy(Strategy):
    bb_sigma2 = 2
    bb_sigma3 = 3
    bb_window = 9
    bb_matype = 0
    macd_fastperiod = 6
    macd_slowperiod = 13
    macd_signalperiod = 4
    
    profit = 40 * 0.01 # 利確30pips
    loss = 20 * 0.01 # 損切り15pips
    
    def init(self):
        self.bb2_upper, self.bb2_lower = self.I(get_bband, self.data.Close, self.bb_window, self.bb_sigma2, self.bb_sigma2, self.bb_matype)
        self.bb3_upper, self.bb3_lower = self.I(get_bband, self.data.Close, self.bb_window, self.bb_sigma3, self.bb_sigma3, self.bb_matype)
        self.macd, self.signal = self.I(get_macd, self.data.Close, self.macd_fastperiod, self.macd_slowperiod, self.macd_signalperiod)
        
        # 0: 上昇トレンド、 1: 下降トレンド、 99: トレンドなし
        self.trand = 99
        
        # トレンドが変わってからのロウソク数
        self.trand_change = 0

    def next(self):
        # macdのトレンド確認
        if (crossover(self.signal, self.macd)):
            self.trand = 0 if (self.signal > self.macd) else 1
        else:
            self.trand_change += 1
        
        # 5本をすぎると騙しが増えるのでトレンドとは考えない
        if (self.trand_change < 5):
            if (crossover(self.data.Close, self.bb2_upper)):
                self.buy(price=self.data.Close[-1], sl=self.data.Close[-1] - self.loss, tp=self.data.Close[-1] + self.profit)
            elif crossover(self.data.Close, self.bb2_lower):
                self.sell(price=self.data.Close[-1], sl=self.data.Close[-1] + self.loss, tp=self.data.Close[-1] - self.profit)
        else:
            self.trand_change = 0
            self.trand = 99
            

In [55]:
bt = Backtest(df, MacdBbandStrategy, cash=100000, commission=.00004)
output = bt.run()
print(output)

Start                     2020-01-30 04:30:00
End                       2020-02-24 13:45:00
Duration                     25 days 09:15:00
Exposure [%]                           94.009
Equity Final [$]                       101077
Equity Peak [$]                        101268
Return [%]                            1.07688
Buy & Hold Return [%]                 2.21371
Max. Drawdown [%]                    -1.07971
Avg. Drawdown [%]                    -0.17579
Max. Drawdown Duration       19 days 02:45:00
Avg. Drawdown Duration        1 days 04:44:00
# Trades                                  263
Win Rate [%]                          50.5703
Best Trade [%]                       0.363594
Worst Trade [%]                     -0.187706
Avg. Trade [%]                     0.00400026
Max. Trade Duration           2 days 03:20:00
Avg. Trade Duration           0 days 02:11:00
Expectancy [%]                      0.0569644
SQN                                  0.840082
Sharpe Ratio                      

In [67]:
stats = bt.optimize(
    bb_window = range(5, 16),
    macd_fastperiod = range(4, 9),
    macd_slowperiod = range(11, 16),
    macd_signalperiod = range(2, 7)
)

/Users/kataoka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Searching best of 1375 configurations.
  """


In [68]:
print(stats)

Start                                                   2020-01-30 04:30:00
End                                                     2020-02-24 13:45:00
Duration                                                   25 days 09:15:00
Exposure [%]                                                        90.4117
Equity Final [$]                                                     103377
Equity Peak [$]                                                      103552
Return [%]                                                          3.37684
Buy & Hold Return [%]                                               2.21371
Max. Drawdown [%]                                                  -0.81308
Avg. Drawdown [%]                                                -0.0961945
Max. Drawdown Duration                                     13 days 02:40:00
Avg. Drawdown Duration                                      0 days 09:13:00
# Trades                                                                184
Win Rate [%]

In [69]:
print(stats._strategy)

MacdBbandStrategy(bb_window=7,macd_fastperiod=7,macd_slowperiod=11,macd_signalperiod=4)
